# 202001549 장희권

In [23]:
from nltk.tokenize import word_tokenize
import nltk

from urllib.request import urlretrieve

from konlpy.tag import Okt
okt = Okt()

In [24]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\82107\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
def pos_tokenize(raw_sent):
    pos_sent = []

    sent = okt.pos(raw_sent, norm=True, stem=True)
    for tup in sent:
        word, tag = tup
        word_tag = word + '/' + tag
        pos_sent.append(word_tag)

    return ' '.join(pos_sent)

In [26]:
def load_data(filename, data_count):
    train = []
    with open(filename, 'r', encoding='utf-8') as f:
        count = 0
        for line in f.readlines():
            if count == data_count:
                break
            count += 1

            line = line.strip()
            id, document, label = line.split('\t')
            if label == '1': label = 'pos'
            else: label = 'neg'
            train_tup = (document, label)
            train.append(train_tup)
    return train

In [27]:
def make_word_dict(train, use_morph=False):
    all_words = set()

    for tup in train:
        sent, label = tup
        if use_morph: sent = pos_tokenize(sent)
        word_list = word_tokenize(sent)
        for word in word_list:
            all_words.add(word)

    return all_words

In [28]:
def make_train_feats(train, all_words, use_morph=False):
    train_features = []

    for tup in train:
        sent, label = tup
        if use_morph: sent = pos_tokenize(sent)
        word_list = word_tokenize(sent)
        tmp = {set_word: set_word in word_list for set_word in all_words}
        train_feature = (tmp, label)
        train_features.append(train_feature)

    return train_features

# count가 500일 때

In [29]:
date_count = 500
train = load_data('ratings_train.txt', date_count)
train = train[1:]

In [30]:
use_morph = True

all_words = make_word_dict(train, use_morph)
train_features = make_train_feats(train, all_words, use_morph)

In [31]:
classifier = nltk.NaiveBayesClassifier.train(train_features)
classifier.show_most_informative_features()

Most Informative Features
                       ; = True              neg : pos    =      8.2 : 1.0
          재미없다/Adjective = True              neg : pos    =      8.2 : 1.0
                주인공/Noun = True              neg : pos    =      7.5 : 1.0
                 최고/Noun = True              pos : neg    =      6.9 : 1.0
                  뭐/Noun = True              neg : pos    =      6.8 : 1.0
           재밌다/Adjective = True              pos : neg    =      6.5 : 1.0
                 내용/Noun = True              neg : pos    =      6.2 : 1.0
       ㅡㅡ/KoreanParticle = True              neg : pos    =      6.1 : 1.0
                스토리/Noun = True              neg : pos    =      6.1 : 1.0
                 다시/Noun = True              pos : neg    =      5.9 : 1.0


In [34]:
urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/" + "ratings_test.txt", "ratings_test.txt")

# ratings_test.txt를 처리하는 부분 작성
test = load_data('ratings_test.txt', date_count)
test_feature = make_train_feats(train, all_words, use_morph)
nltk.classify.accuracy(classifier, test_feature)

0.9579158316633266

# count가 1000일 때

In [36]:
date_count = 1000
train = load_data('ratings_train.txt', date_count)
train = train[1:]

In [37]:
use_morph = True

all_words = make_word_dict(train, use_morph)
train_features = make_train_feats(train, all_words, use_morph)

In [38]:
classifier = nltk.NaiveBayesClassifier.train(train_features)
classifier.show_most_informative_features()

Most Informative Features
                쓰레기/Noun = True              neg : pos    =     11.9 : 1.0
                 인생/Noun = True              pos : neg    =     10.0 : 1.0
                 최고/Noun = True              pos : neg    =      9.5 : 1.0
           괜찮다/Adjective = True              pos : neg    =      8.6 : 1.0
                       ; = True              neg : pos    =      8.3 : 1.0
          재미없다/Adjective = True              neg : pos    =      8.1 : 1.0
       ㅡㅡ/KoreanParticle = True              neg : pos    =      8.1 : 1.0
           아깝다/Adjective = True              neg : pos    =      7.6 : 1.0
          지루하다/Adjective = True              neg : pos    =      7.5 : 1.0
           재밌다/Adjective = True              pos : neg    =      7.5 : 1.0


In [39]:
urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/" + "ratings_test.txt", "ratings_test.txt")

# ratings_test.txt를 처리하는 부분 작성
test = load_data('ratings_test.txt', date_count)
test_feature = make_train_feats(train, all_words, use_morph)
nltk.classify.accuracy(classifier, test_feature)

0.973973973973974

데이터가 많으면 많을수록 학습량이 커져 정확도가 높아진다.